In [22]:
import pandas as pd
import numpy as np

# import the required classes and definitions from Python-MIP
from mip import Model, xsum, maximize, BINARY, INTEGER

In [23]:
#sort the top500 by element_type so model can fill correct number of each type

top500 = pd.read_csv('top500.csv')
top500sorted = top500.sort_values('element_type')
fitted = pd.read_csv('fitted')
top500sorted = top500sorted.merge(fitted, how='left', on='id_')
top500sorted.fillna(0, inplace=True)

#lists for the dream team model, sorted by position
eltype = list(top500sorted['element_type'])
form = list(top500sorted['form'])
score = list(top500sorted['total_points'])
price = list(top500sorted['now_cost'])
team = list(top500sorted['team'])
fitted = list(top500sorted['fitted'])
form_score = list(pd.Series(form) * pd.Series(score))


In [24]:
#make a list of our main columns of interest
simple_columns = ['web_name', 'element_type', 'total_points', 'fitted', 'now_cost', 'form',
                 'goals_scored', 'assists', 'clean_sheets', 'team', 'selected_by_percent']

#make a smaller df for easy viewing of dream_team
top500simple = top500sorted[simple_columns]
top500simple.shape

(500, 11)

In [25]:
#find cutoffs for the element_types

eltype_counts = top500sorted['element_type'].value_counts().sort_index()

cutoff1 = eltype_counts[1]
cutoff2 = cutoff1 + eltype_counts[2]
cutoff3 = cutoff2 + eltype_counts[3]
cutoff4 = cutoff3 + eltype_counts[4]

print(cutoff1, cutoff2, cutoff3, cutoff4)
eltype_counts

48 221 431 500


1     48
2    173
3    210
4     69
Name: element_type, dtype: int64

In [26]:
df = top500sorted
type(df)

pandas.core.frame.DataFrame

In [27]:
df = df.reset_index()
df.head()

,index,id_,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,...,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,fitted
0,0,427,0,0,0,0.0,0.0,0,98980,-1,...,0,4392,108,11499,229,0.0,0.0,Martínez,0,0.488977
1,1,47,0,11,362,0.0,0.0,4,131897,0,...,65,843829,26075,483019,13432,0.7,13.5,Ryan,0,2.998547
2,2,449,0,0,6,100.0,100.0,0,28082,0,...,1,4334,96,23625,251,0.0,0.2,Fahrmann,0,0.488977
3,3,411,0,4,271,0.0,0.0,4,38533,0,...,62,803492,6450,425299,27336,0.6,11.9,Patrício,0,2.924116
4,4,131,0,7,275,100.0,100.0,6,40836,1,...,61,156504,56175,54364,2970,1.4,12.2,Guaita,1,2.878590


In [28]:
ars = list(df[df.team == 1].index)
avl = list(df[df.team == 2].index)
bou = list(df[df.team == 3].index)
bha = list(df[df.team == 4].index)
bur = list(df[df.team == 5].index)
che = list(df[df.team == 6].index)
cry = list(df[df.team == 7].index)
eve = list(df[df.team == 8].index)
lei = list(df[df.team == 9].index)
liv = list(df[df.team == 10].index)
mci = list(df[df.team == 11].index)
mun = list(df[df.team == 12].index)
new = list(df[df.team == 13].index)
nor = list(df[df.team == 14].index)
shu = list(df[df.team == 15].index)
sou = list(df[df.team == 16].index)
tot = list(df[df.team == 17].index)
wat = list(df[df.team == 18].index)
whu = list(df[df.team == 19].index)
wol = list(df[df.team == 20].index)

In [29]:
# The model

# define the problem data, change the profit list here
profit = fitted
weight = price
c = 830
n = len(weight)

#create an empty maximization
m = Model('team_select')

#add the binary decision variables to model m and store their references in a list x
x = [m.add_var(var_type=BINARY) for i in range(n)]

#define the objective function of this model 
m.objective = maximize(xsum(profit[i] * x[i] for i in range(n)))

#add the capacity constraints
m += xsum(weight[i] * x[i] for i in range(n)) <= c

m += xsum(x) <= 11

m += xsum(eltype[i] * x[i] for i in range(cutoff1)) >= 1
m += xsum(eltype[i] * x[i] for i in range(cutoff1)) <= 1

m += xsum(eltype[i] * x[i] for i in range(cutoff1,cutoff2)) >= 6
m += xsum(eltype[i] * x[i] for i in range(cutoff1,cutoff2)) <= 10

m += xsum(eltype[i] * x[i] for i in range(cutoff2,cutoff3)) >= 6
m += xsum(eltype[i] * x[i] for i in range(cutoff2,cutoff3)) <= 15

m += xsum(eltype[i] * x[i] for i in range(cutoff3,cutoff4)) >= 4
m += xsum(eltype[i] * x[i] for i in range(cutoff3,cutoff4)) <= 12


# max 3 per team

m += xsum(df.team[i] * x[i] for i in ars) <= 3
m += xsum(df.team[i] * x[i] for i in avl) <= 6
m += xsum(df.team[i] * x[i] for i in bou) <= 9
m += xsum(df.team[i] * x[i] for i in bha) <= 12
m += xsum(df.team[i] * x[i] for i in bur) <= 15
m += xsum(df.team[i] * x[i] for i in che) <= 18
m += xsum(df.team[i] * x[i] for i in cry) <= 21
m += xsum(df.team[i] * x[i] for i in eve) <= 24
m += xsum(df.team[i] * x[i] for i in lei) <= 27
m += xsum(df.team[i] * x[i] for i in liv) <= 30
m += xsum(df.team[i] * x[i] for i in mci) <= 33
m += xsum(df.team[i] * x[i] for i in mun) <= 36
m += xsum(df.team[i] * x[i] for i in new) <= 39
m += xsum(df.team[i] * x[i] for i in nor) <= 42
m += xsum(df.team[i] * x[i] for i in shu) <= 45
m += xsum(df.team[i] * x[i] for i in sou) <= 48
m += xsum(df.team[i] * x[i] for i in tot) <= 51
m += xsum(df.team[i] * x[i] for i in wat) <= 54
m += xsum(df.team[i] * x[i] for i in whu) <= 57
m += xsum(df.team[i] * x[i] for i in wol) <= 60



#Optimize the model
m.optimize()

# Compute the solution, a list of the selected items
selected = [i for i in range(n) if x[i].x >= 0.99]
print('selected items: {}'.format(selected))
dream_team = top500simple.iloc[selected,:]
dream_team

selected items: [14, 93, 204, 217, 242, 253, 278, 291, 420, 437, 491]


,web_name,element_type,total_points,fitted,now_cost,form,goals_scored,assists,clean_sheets,team,selected_by_percent
14,Pope,1,56,3.395241,47,2.8,0,0,5,5,14.6
93,Maitland-Niles,2,19,2.886546,46,0.5,0,2,1,1,2.6
204,Pieters,2,34,2.879174,45,0.0,0,3,2,5,6.6
217,Lundstram,2,78,4.201036,51,2.0,3,2,5,15,47.1
242,Ndidi,3,48,4.432330,50,2.2,2,0,7,9,2.8
253,Martial,3,52,4.514689,76,4.8,4,3,3,12,5.8
278,Sterling,3,81,4.874876,118,3.2,8,1,5,11,24.8
291,Mané,3,109,4.876623,123,6.6,9,6,3,10,42.7
420,Lamela,3,32,4.760647,57,0.0,2,2,1,17,1.2
437,Pukki,4,84,7.704726,66,5.8,8,3,2,14,17.7


In [25]:
sum(dream_team['now_cost'])

818

In [26]:
sum(dream_team['total_points'])

786